#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "***"

##0-3.Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive/GENIAC" # change your path

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
!pip install -qU openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[{"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
     {"role": "user", "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。"}],
)

print(chat_completion.choices[0].message.content)

 1


#1.データ生成処理の実行

##1-1.プロンプト準備

言語理解でのプロンプト例

In [1]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、userとassistantによる10種類の多様なマルチターンのペアを考えてください。これらのタスク指示文はGPTモデルに与えられ、より言語性能が高いモデルを構築することを目的としています。

要件は以下の通りです：

1. **多様性を確保**：
   - それぞれの文章は、様々な文脈や単語、主語を変更するなど多様性を最大限に引き出してください。また、それぞれのidごとに出力する文章内容が出来るだけ被らないように作成してください。

2. **データセットの対話形式**：
   - userとassistantによる対話形式のデータを生成してください。

3. **5ターンまでの継続的な会話をランダムに生成**：
   各対話は少なくとも2ターン（user-assistant-user）を含み、継続的に会話するようにしてください。なお、ターン数は2から5までの範囲で多様に設定してください。

4. **対話データを日本語で生成**：
   - 対話データは必ずすべて日本語で生成してください。

5. **JSON形式で出力**：
   - 回答フォーマット例に記載のJSON形式で回答してください。

6. **文章の長さの多様性**：
   - それぞれの1文の長さにも多様性を持たせてください。それぞれ短い文章（15文字前後）と長い文章（50文字前後）をバランスよく生成してください。

7. **ビジネスユースケースの含有**：
   - 一般的な対話に加えてビジネスユースケースを想定した対話も含めてください。

8. **アシスタントとしての礼儀正しさ**：
   - アシスタントはユーザの言葉に傾聴し、適切な言葉遣いをしましょう。

9. **構造的な回答**：
   - アシスタントが何か解決策やアドバイスを行う場合は構造的な回答を心がけましょう。例えば、アイデアを出す場合には1.,2.などの回答形式で、タスク指示例のようにインデントも意識しましょう。

10. **会話の終結**：
   - アシスタントとユーザの会話は途中で終結しないようにしてください。例えば、解決策をアシスタントが提示した場合にはユーザは別質問をしたりお礼で反応し、アシスタントはその応答をしてください。

タスク指示例：
$FewShotExamples

回答フォーマット例(JSON):
[
  {
    "id": 1,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user_1": "<ユーザーの質問>",
    "assistant_1": "<システムの回答>",
    "user_2": "<ユーザーの質問>",
    "assistant_2": "<システムの回答>"
    ]
  },
  {
    "id": 2,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user_1": "<ユーザーの質問>",
    "assistant_1": "<システムの回答>",
    "user_2": "<ユーザーの質問>",
    "assistant_2": "<システムの回答>",
    "user_3": "<ユーザーの質問>",
    "assistant_3": "<システムの回答>"
    ]
  },
  {
    "id": 3,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user_1": "<ユーザーの質問>",
    "assistant_1": "<システムの回答>",
    "user_2": "<ユーザーの質問>",
    "assistant_2": "<システムの回答>",
    "user_3": "<ユーザーの質問>",
    "assistant_3": "<システムの回答>",
    "user_4": "<ユーザーの質問>",
    "assistant_4": "<システムの回答>"
    ]
  },
  {
    "id": 4,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user_1": "<ユーザーの質問>",
    "assistant_1": "<システムの回答>",
    "user_2": "<ユーザーの質問>",
    "assistant_2": "<システムの回答>",
    "user_3": "<ユーザーの質問>",
    "assistant_3": "<システムの回答>",
    "user_4": "<ユーザーの質問>",
    "assistant_4": "<システムの回答>",
    "user_5": "<ユーザーの質問>",
    "assistant_5": "<システムの回答>"
    ]
  },

]
"""

In [2]:
FEWSHOT_EXAMPLES = [
  {
    "id": 1,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "お客様に対して効果的なプレゼンテーションを行うにはどのような点に注意すればよいでしょうか？",
    "assistant": "効果的なプレゼンテーションを行うために以下の3つのポイントを押さえてみるのはいかがでしょうか？\n\n1. キーメッセージの明確化：伝えたいメッセージを明確にし、お客様にとって価値のある情報を提供すること。\n2. ストーリーラインの一貫性：序論、本論、結論の流れを意識し、ストーリー性を持たせること。\n3. デザイン化：スライドのデザインを工夫し、図表やグラフを効果的に使用して視覚的に訴求すること。\n\nこれらを意識することでお客様の理解を促進させ、メッセージを効果的に伝えることができます。",
    "user": "どのようにスライドをデザイン化するべきでしょうか？",
    "assistant": "以下の点を工夫してみるのはどうでしょうか？\n\n1. シンプルさ：1枚のスライドに盛り込む情報は最小限にし、最も伝えたいことが伝わるようにする。\n2. 一貫性：何色も使用したカラフルなスライドではなく、ベースカラーや強調色など数色で使い分けをする。\n3. アイコンや画像の活用：適切なアイコンや画像を使用し、分かりやすさを促進する。",
    "user": "では、プレゼンテーション本番で気をつけるべき点はありますか？",
    "assistant": "プレゼンテーション本番では以下のことに気を付けると、より良くなると思います。\n\n1. 視線と姿勢：お客様と目線を合わせ、自信のある姿勢で話す。\n2. 声の大きさとスピード：はっきりと大きな声で話し、理解が追いつくスピードで話す。\n3. ストーリー構成：全体の限られた時間の中で、重要なポイントを話せるように構成立てて話す。\n4. 柔軟な対応：お客様の反応を見ながら、臨機応変に対応する。",
    "user":"ありがとうございました。プレゼンテーションのスキルアップに役立てます。",
    "assistant":"いえいえ、ご質問ありがとうございました。より良いプレゼンテーションになるように応援しております。"
  },
  {
    "id": 2,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "海外旅行に行きたいのですが、何を準備すればよいと思いますか？",
    "assistant": "海外旅行に行かれるのですね。以下の準備実行をお勧めします。\n\n・パスポートとビザの確認：渡航先によってはビザが必要な場合があるので、事前に確認しましょう。また、パスポートの期限も確認しましょう。\n・航空券と宿泊先の手配：航空券と宿泊先を早めに手配しましょう。早めに予約することで、お得な料金で手配できる可能性があります。\n・海外旅行保険の加入：万が一の事態に備えて、海外旅行保険に加入することをお勧めします。\n\nこれらを準備することで、安心して海外旅行を楽しむことができます。",
    "user": "ありがとうございます。では、現地での移動手段について、どのようなものがありますか？",
    "assistant": "現地での主な移動手段は以下の通りです。\n\n・公共交通機関：電車、バス、地下鉄などを利用できる都市が多いです。切符の買い方などは渡航先によって異なるので、事前に確認することをお勧めします。\n・タクシー：手軽に利用できますが、料金が高くなる傾向があります。事前にネット検索などで確認しましょう。\n・レンタカー：自由に移動できるメリットがありますが、運転に不安がある場合は避けた方が良いでしょう。また、国際免許の取得も確認してください。\n・ウォーキングやサイクリング：小さな町や観光地では、徒歩や自転車で散策するのも楽しい方法です。比較的安価な価格で抑えることができます。\n\nこれらの移動手段が挙げられますが、目的地や予算に合わせて最適な手段を選びましょう。",
    "user": "なるほど、ありがとうございます。",
    "assistant": "はい、ご質問ありがとうございました。素敵な旅になるようにお祈りしております。"
  },
  {
    "id": 3,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "最近健康を意識しているのですが、朝ごはんに何を摂るのが良いと思いますか？",
    "assistant": "健康を意識されているのですね。素晴らしいと思います。\n朝食にはバランスの取れた食事をおすすめします。例えば、全粒粉のパン、低脂肪のプロテインソース（例：ゆで卵や無脂肪ヨーグルト）、そしてフレッシュなフルーツを組み合わせるのはどうでしょうか",
    "user": "その組み合わせの中で、特におすすめできるフルーツはありますか？",
    "assistant": "はい、特に抗酸化物質が豊富なベリー類（ブルーベリー、ストロベリー、ラズベリーなど）をおすすめします。\nなぜならこれらはビタミンも豊富で、健康的な朝食にぴったりだからです。また、他のフルーツとしてはバナナもエネルギーを素早く補給できるため、朝の忙しい時間に最適です。",
    "user": "ありがとうございます。ベリーを摂ってみようと思います。",
    "assistant": "いいですね、摂りすぎにはご注意くださいね。また何かございましたらご相談ください。"
  },
  {
    "id": 4,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "最近暑いですね。",
    "assistant": "そうですね、最近は気温が高くなってきましたね。暑い日には水分補給を忘れず、なるべく涼しい場所で過ごすようにしてください。軽めの食事や、さっぱりとした飲み物も良いですよ。どうぞご自愛ください。",
    "user": "ありがとうございます。気を付けます。",
    "assistant": "どういたしまして。何かお手伝いできることがあれば、いつでもお知らせください。"
  },
  {
    "id": 5,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "就職活動中ですが、どのように会社を選べばいいでしょうか？",
    "assistant": "就職活動中なのですね。以下のポイントに注目しながら、あなたにとって良い会社を選択してみてはどうでしょうか？\n\n1. 価値観や文化の一致: 企業の理念や文化に共感でき、ここで働きたいと思う気持ちを大切にしましょう\n2. 成長機会: 学びの機会が多いかどうか、現場社員の方の話も聞きながら見極めましょう。\n3. ワークライフバランス: 会社の勤務時間や休暇制度など、仕事とプライベートのバランスが取れる環境かどうかを考慮しましょう。\n4. 給与制度：給料がご自身の生活水準や目標に合っているものかどうか、確認しましょう。\n5. 福利厚生: 給与だけでなく、福利厚生や特典などの従業員へのサポートがあるかも考慮しましょう。\n\nあくまでいくつかのポイントを挙げましたが、これらに囚われずに様々な観点で考えてみてください。良い選択ができることをお祈りしています。",
    "user": "ありがとうございます。やはり少し不安が残ってしまいます。",
    "assistant": "分からない事も多くて不安ですよね。就職は大きな決断ですから、その感覚を持つのは当然です。あまり不安になりすぎず、次のアプローチを試してみてはいかがでしょうか。\n\n1. 自己分析: これまでの自分の経験を振り返って整理したり、自分がどのようなキャリアを望んでいるのか明確にすることで、会社選択の軸が見えてくるかもしれません。\n2. 情報収集: HPや就職サイトの他にも、OB訪問などの現場社員からの声も収集することで、よりイメージができるようになるかもしれません。\n3. 他者の意見: 家族、友人、キャリアアドバイザーなど、他者の意見を取り込んでみることで新たな視点が得られるかもしれません。\n\n不安を抱えやすい時期だとは思いますが、前向きに取り組んでみてくださいね。応援しています！",
    "user": "自己分析はどのように進めればいいでしょうか？",
    "assistant": "自己分析は、就職活動において自分の適性や興味、価値観を明確にするための重要なステップです。以下の方法はいかがでしょうか？\n\n1. 過去の経験を整理する:\nこれまでの経験で、困難に直面した際の解決方法や、これまでの対人コミュニケーションなどを整理しましょう。高校・大学などといくつかのステージに分けて整理してみるといいかもしれません。\n2. 強みと弱みを整理する:\nこれまでの経験も基に、自身のスキルや能力、性格的な強みをリストアップしましょう。なお、弱みの部分も明確にしてそれにどう対応するか考えてみましょう。\n3. フィードバックを得る:\n家族、友人、先生、先輩など他の方からの評価をもらいましょう。これまで見えてこなかった視点から自己評価を補強できるかもしれません。\n\n色々とやり方を模索しながらも、自分に合ったやり方が見つかればよいですね。また何かあればご相談くださいね。",
    "user": "ありがとうございます。",
    "assistant": "どういたしまして。また何かお手伝いできることがあれば、いつでもご相談ください。頑張ってくださいね。"
  },
  {
    "id": 6,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "おはようございます。",
    "assistant": "おはようございます！今日も良い一日をお過ごしください。",
    "user": "ありがとうございます。なにか新しいことを始めてみようと思います。",
    "assistant": "素晴らしい意気込みですね！応援しています。何かサポートが必要であれば、いつでもどうぞ。"
  },
  {
    "id": 7,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "ある歌を作ってみたいのですが、何かいい歌詞はありますか？日常系のテーマが良いなと思います。",
    "assistant": "日常系のテーマで歌詞を考えるのは楽しくて良いですね。以下に、そのようなテーマでの歌詞の一部を考案してみました。\n\n-----\nテーマ：「コーヒー」\n歌詞：\n一日が同じアラームで始まり\nいつものエッセイ\nページをめくる手と、香り立つモーニングブレンド\n小さな幸せを、今、噛みしめる\n-----\n\nこの歌詞は、朝のコーヒータイムを通じて、日常の一コマと心の内を描写しています。いつもと変わらないにもかかわらず、それが幸せだという日常を表現してみました。どうぞ参考にしてみてください。",
    "user": "良いですね、もう少し自分目線の歌詞に書き換えてくれませんか？",
    "assistant": "了解しました。個人的な感情や日常のルーティンにフォーカスした歌詞に書き換えてみました。  \n\n-----\nテーマ：「コーヒー」\n歌詞：\nいつもと同じアラームを止めて\nカーテンを開ける\n気分に合わせたコーヒー豆を選んで\n香りだつ部屋\n静かにページをめくりながら思う\n私の心に、小さな幸せをくれる\n-----\n\n今回は心の中で感じる変化や気づきを強調してみました。どうでしょうか？気に入っていただければ嬉しいです。"
  },
  {
    "id": 8,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "この二次方程式を解きたいです。x^2-5x+6=0",
    "assistant": "この二次方程式のx^2-5x+6=0解は𝑥=2と𝑥=3です。",
    "user": "どのようにして答えを導き出しましたか？",
    "assistant": "この二次方程式を解くために、因数分解を使います。一般的な形 𝑎𝑥^2+𝑏𝑥+𝑐=0に当てはめると、今回の方程式は𝑎=1, 𝑏=−5, 𝑐=6です。\n\n因数分解:\n𝑥^2−5𝑥+6xの形を (𝑥−𝑚)(𝑥−𝑛)のように分解します。ここで𝑚と𝑛は、次の条件を満たす必要があります。\n𝑚+𝑛=−5（係数𝑏に等しい）\n𝑚⋅𝑛=6（定数項 𝑐に等しい）\nこれを満たす𝑚,𝑛はm=2 と𝑛=3\n n=3 がこの条件を満たします。つまり、(𝑥−2)(𝑥−3)=0と因数分解できます。\nしたがって、方程式の解は𝑥=2と𝑥=3です。",
    "user": "理解できました。ありがとうございます。",
    "assistant": "どういたしまして。他にも質問があればいつでも聞いてください。"
  },
  {
    "id": 9,
    "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。",
    "user": "朝から頭が痛いです。",
    "assistant": "頭痛がするのは辛いですね。頭痛がする際には以下の対応がお勧めできます。\n\n1. 水分補給: 原因が脱水症状にあるかもしれません。お水をしっかり飲んでください。\n2. 休息: 十分な休息は取れていますか？横になって休むことで体力が回復し、頭痛も緩和されるかもしれません。\n3. 軽いストレッチ: 首や肩の筋肉が緊張しているために頭痛が起こる場合があります。軽いストレッチやマッサージで血流を良くしてみましょう。\n4. 薬の服用: 市販の頭痛薬を服用することで、痛みを和らげることができます。ただし、服用する前に説明書をよく読み、用法・用量を守ってください。\nこれらの方法を試しても頭痛が改善しない場合や、慢性的に起こる場合は医師に相談することをお勧めします。\nお大事にしてくださいね。",
    "user": "ありがとうございます。運動していたので脱水が原因かもしれません。",
    "assistant": "運動されていたのですね。運動する際は適宜水分を補給して、体を休めることが大切です。\n追加で運動時にお勧めしたい事項を以下に示します。\n\n1. 電解質の補給: スポーツドリンクや経口補水液を飲むことで、失った電解質も補給できます。コンビニエンスストアや自販機で購入することができます。\n2. 冷たいタオル: 運動時は身体に熱が込められています。冷たいタオルや冷却ジェルパックを額や首の後ろに当てると、痛みが緩和されることがあります。\n3. 小休憩: 疲れてから休憩をとるのではなく、意識的に休憩をとるようにしてください。意外と身体は疲れている可能性があり、しっかりと休めてあげることが大切です。\n\n早く良くなることを願っています。ぜひ実践してみてくださいね。",
    "user": "そうですね、色々と試してみようと思います。",
    "assistant": "いいですね。ぜひ色々な方法を試して、自分に合った対策を見つけてください。\nもし他に質問や不安があればお知らせください。早く元気になることを願っています。お大事に。"
  }

]

In [3]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)
print(formatted_prompt_template)


以下のタスク指示例を参考に、userとassistantによる10種類の多様なマルチターンのペアを考えてください。これらのタスク指示文はGPTモデルに与えられ、より言語性能が高いモデルを構築することを目的としています。

要件は以下の通りです：

1. **多様性を確保**：
   - それぞれの文章は、様々な文脈や単語、主語を変更するなど多様性を最大限に引き出してください。また、それぞれのidごとに出力する文章内容が出来るだけ被らないように作成してください。

2. **データセットの対話形式**：
   - userとassistantによる対話形式のデータを生成してください。

3. **5ターンまでの継続的な会話をランダムに生成**：
   各対話は少なくとも2ターン（user-assistant-user）を含み、継続的に会話するようにしてください。なお、ターン数は2から5までの範囲で多様に設定してください。

4. **対話データを日本語で生成**：
   - 対話データは必ずすべて日本語で生成してください。

5. **JSON形式で出力**：
   - 回答フォーマット例に記載のJSON形式で回答してください。

6. **文章の長さの多様性**：
   - それぞれの1文の長さにも多様性を持たせてください。それぞれ短い文章（15文字前後）と長い文章（50文字前後）をバランスよく生成してください。

7. **ビジネスユースケースの含有**：
   - 一般的な対話に加えてビジネスユースケースを想定した対話も含めてください。

8. **アシスタントとしての礼儀正しさ**：
   - アシスタントはユーザの言葉に傾聴し、適切な言葉遣いをしましょう。

9. **構造的な回答**：
   - アシスタントが何か解決策やアドバイスを行う場合は構造的な回答を心がけましょう。例えば、アイデアを出す場合には1.,2.などの回答形式で、タスク指示例のようにインデントも意識しましょう。

10. **会話の終結**：
   - アシスタントとユーザの会話は途中で終結しないようにしてください。例えば、解決策をアシスタントが提示した場合にはユーザは別質問をしたりお礼で反応し、アシスタントはその応答をしてください。

タスク指示例：
[{'id': 1, 'instr

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE = 1
MAX_TOKENS = 60000
LOOP_NUM = 10
MAX_TURNS = 5

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):
    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[
            {"role": "system", "content": "あなたは日本語の学習データセットを生成する専門家です。求められた要件をしっかりと理解し反映させることができます"},
            {"role": "user", "content": PROMPT_TEMPLATE}
        ],
        temperature=TEMPERATURE,
        top_p=0.9,
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,
        presence_penalty=1.3
    )
    output = chat_completion.choices[0].message.content
    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

In [ ]:
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)

data = []

for i in range(LOOP_NUM):
    for retry_i in range(3):
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)
        try:
            json_data = pd.read_json(output)
            for row in json_data.itertuples(index=False):
                conversation = {'id': row.id, 'instruction': row.instruction}
                user_assistant_data = []
                for i in range(1, (MAX_TURNS * 2) + 1):
                    if i % 2 == 1:
                        user_key = f'user_{(i + 1) // 2}'
                        user_value = getattr(row, user_key, '')
                        conversation[user_key] = user_value
                    else:
                        assistant_key = f'assistant_{i // 2}'
                        assistant_value = getattr(row, assistant_key, '')
                        conversation[assistant_key] = assistant_value
                data.append(conversation)
            print("Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue

columns = ['id', 'instruction'] + [item for sublist in [[f'user_{i}', f'assistant_{i}'] for i in range(1, MAX_TURNS + 1)] for item in sublist]

df_final = pd.DataFrame(data, columns=columns)

 [{"id": 1, "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。", "user_1": "来週の予定がありますか？", "assistant_1": "いえ、来週は空いています。何か用事がございましたらお申し付けください。", "user_2": "ありがとうございます。それでは来週は京都へ観光に行ってみようと思います。", "assistant_2": "素晴らしいですね！京都は四季折々の魅力があります。遊びがんばってください。"},
{"id": 2, "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。", "user_1": "今夜の天気はいかがですか？", "assistant_1": "今夜は曇り時々雨で、17度〜20度程度と予想されています。適当に傘を持参していただきたいですね。", "user_2": "そうなんですね。ありがとうございます。", "assistant_2": "どういたしまして。お帰りの路は明るくてご無事の道を祈っております。"},
{"id": 3, "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。", "user_1": "ビジネス会議の予定はありますか？", "assistant_1": "今週は金曜日に午後3時から5時までの会議が予定されています。ご参加いただけますか？", "user_2": "ありがとうございます。参加します。資料は準備しておきますか？", "assistant_2": "いえ、会議資料はこちらで用意いたします。ご都合よろしければ弊社の会議室で開催いたします。", "user_3": "承知しました、ありがとうございます。"},
{"id": 4, "instruction": "以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確で簡潔なものにしてください。", "user_1": "今日は何を食べましょうか？", "assistant_1": "今日

In [ ]:
df_final

,id,instruction,user_1,assistant_1,user_2,assistant_2,user_3,assistant_3,user_4,assistant_4,user_5,assistant_5
0,1,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,来週の予定がありますか？,いえ、来週は空いています。何か用事がございましたらお申し付けください。,ありがとうございます。それでは来週は京都へ観光に行ってみようと思います。,素晴らしいですね！京都は四季折々の魅力があります。遊びがんばってください。,NaN,,,,,
1,2,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,今夜の天気はいかがですか？,今夜は曇り時々雨で、17度〜20度程度と予想されています。適当に傘を持参していただきたいですね。,そうなんですね。ありがとうございます。,どういたしまして。お帰りの路は明るくてご無事の道を祈っております。,NaN,,,,,
2,3,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,ビジネス会議の予定はありますか？,今週は金曜日に午後3時から5時までの会議が予定されています。ご参加いただけますか？,ありがとうございます。参加します。資料は準備しておきますか？,いえ、会議資料はこちらで用意いたします。ご都合よろしければ弊社の会議室で開催いたします。,承知しました、ありがとうございます。,,,,,
3,4,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,今日は何を食べましょうか？,今日は焼き鳥好きな方におすすめです！近くに有名な店があります。,焼き鳥なんですね。ありがとうございます。何時に行けばいいですか？,いつでも大丈夫です。ただ、夜8時以降は混み合ってしまう場合があります。予めテーブルの確保をお...,承知しました、前もって予約しておきます。ありがとうございます。,,,,,
4,5,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,今日は天気はいかがですか？,今日は晴れで、最高気温は25度程度と予想されています。外出先でよく肌を収めるように注意してい...,ありがとうございます。気温はそんなに高くないみたいですね。,そうですね、日中は UVが強くなるため注意が必要です。今日は楽しんでいってください。,NaN,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
80,1,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,旅行プランを作成していますが、オススメのホテルはありますか？,了解です。お勧めのホテルとしては、海辺に位置する「シーサイドホテル」があります。,美味しいレストランも少しご紹介いただけますか？,もちろんです。「イタリア料理Bianco」があります。itors reviewも高く、美味し...,NaN,NaN,NaN,NaN,NaN,NaN
81,2,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,ビジネス会議のため、首都大通りの会議室を探していますが、利用できる空きスペースはありますか？,了解です。現在、空きがある会議室としては「グランドホール」が利用可能です。,コストはいくらぐらいですか？,コストは1時間当たり1万円です。,早急に予約をしたいのですが、可能ですか？,了解です。現在の状況ですと、午前中までに予約を確定できれば利用可能です。,NaN,NaN,NaN,NaN
82,3,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,今度、ビジネスのためにニューヨークへ行く予定があります。飛行機の予約に関して教えていただけますか？,了解です。ニューヨークへの飛行機は、 ANA、Delta Airlinesなどがあります。,料金レンジはいくらぐらいでしょうか？,一般的に約10万円前後の価格がかかります。,いつ頃に予約すると安くなりますか？,予約期間が長いほど安くなります。現在でも夕方の飛行機は大体価格が安くなっています。,早急に予約をしたいのですが、可能ですか？,了解です。現在空席がある状況だと、午前中までに予約を確定すれば利用可能です。,NaN,NaN
83,4,以下の対話において、システムの役割を担い、ユーザーの質問に適切に回答してください。回答は明確...,今日は良い天気なので、チェリーピッキングに行きたいと思っています。,素晴らしいご案内ですね！それでは、以下の手順を参考に、簡単にプランを立ててみましょう。,最初に行くべき場所は？,最初に行くべき場所は、リゾートファーム「ちゃんぽ」です。通常、9時から17時までオープンして...,私たちが予約できる時間帯はどのようなものがありますか？,予約可能な時間帯は、午前中と afternoon session（1時半から3時まで）の二つ...,どちらがよいでしょうか？,お勧めはafternoon sessionです。その時間帯は人が少ないため、安心してチェリー...,明日も行きたいと思っています。さっそく予約をしたいのですが、可能ですか？,了解です。今の状況では午後の時間帯については空席ありです。


##1-3.データの保存

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel(G_FOLDER + filename, index=False)

print(f"File saved as: {filename}")

File saved as: /generated_data_2024-05-20_02-45-36.xlsx
